# Appendix

## MM925: Quantitative Risk Analysis

### Packages Used

Using R version 4.40, Visual Studio Code version 1.90 and IRkernal version 1.3.2. A number of packages within R have been used listed below:

In [75]:
# Set Seed - this assignment does not require student number
set.seed(123)
# Set the plot background color to white
par(bg = "white")
# Turning off errors for readability
options(warn=0)

## Helper Functions

In [76]:
# Core calculation functions
calculate_p <- function(v, mu, t) {
    (v * (1 - exp(-(mu + v) * t))) / (v + mu)
}

# Summary statistics helper
summarize_distribution <- function(values, name) {
    cat(sprintf("\nSummary Statistics for %s:\n", name))
    cat("Mean:", formatC(mean(values), format="e", digits=3), "\n")
    cat("Median:", formatC(median(values), format="e", digits=3), "\n")
    cat("SD:", formatC(sd(values), format="e", digits=3), "\n")
    cat("95% CI: [", 
        formatC(quantile(values, 0.025), format="e", digits=3), ", ",
        formatC(quantile(values, 0.975), format="e", digits=3), "]\n")
}

# Bayesian analysis helpers
calculate_posterior_params <- function(n, s, prior_alpha = 1, prior_beta = 1) {
    list(
        alpha = s + prior_alpha,
        beta = n - s + prior_beta
    )
}

calculate_beta_mode <- function(alpha, beta) {
    if (alpha > 1 && beta > 1) {
        (alpha - 1)/(alpha + beta - 2)
    } else {
        NA  # Mode doesn't exist for some parameter combinations
    }
}

## Question 1

In [ ]:
# Question 1 Data
v_min <- 0.00001
v_max <- 0.0006
v_mode <- 0.000187
mu <- 0.078
t <- 2  # 2 week holiday
n_samples <- 10000

# Normal Distribution Sample Generation
generate_normal_samples <- function(n_samples, v_min, v_max, v_mode) {
    v_sd <- (v_max - v_min) / 6  # Range rule
    v_norm <- rnorm(n_samples, mean = v_mode, sd = v_sd/2)
    # Truncate to bounds - fix the pmin/pmax syntax
    v_norm <- pmin(pmax(v_norm, v_min), v_max)  # This was incorrect before
    return(v_norm)
}

# Beta Distribution Sample Generation
generate_beta_samples <- function(n_samples, v_min, v_max, v_mode, alpha = 20) {
    mode_scaled <- (v_mode - v_min) / (v_max - v_min)
    beta <- alpha * (1 - mode_scaled) / mode_scaled
    
    v_beta <- rbeta(n_samples, alpha, beta)
    # Scale beta samples to the correct range
    v_beta <- v_min + (v_max - v_min) * v_beta
    return(v_beta)
}

v_norm <- generate_normal_samples(n_samples, v_min, v_max, v_mode)
v_beta <- generate_beta_samples(n_samples, v_min, v_max, v_mode)

# Calculate probabilities
p_norm <- calculate_p(v_norm, mu, t)
p_beta <- calculate_p(v_beta, mu, t)

# Create plots
par(mfrow=c(2,2))

# Plot 1: Normal Distribution of ν
hist(v_norm, breaks=50, probability=TRUE, 
     main="Normal Distribution of ν",
     xlab="ν value", 
     ylab="Density",
     col="lightgreen", 
     border="white",
     xlim=c(v_min, v_max))

# Plot 2: Beta Distribution of ν
hist(v_beta, breaks=50, probability=TRUE, 
     main="Beta Distribution of ν",
     xlab="ν value", 
     ylab="Density",
     col="lightblue", 
     border="white",
     xlim=c(v_min, v_max))  

# Plot 3: Normal Distribution of p(t)
hist(p_norm, breaks=50, probability=TRUE,
     main="Normal Distribution of p(t)",
     xlab="Probability of Infection",
     ylab="Density",
     col="lightgreen",
     border="white")
curve(dnorm(x, mean=mean(p_norm), sd=sd(p_norm)), 
      add=TRUE, col="red", lwd=2)

# Plot 4: Beta Distribution of p(t)
hist(p_beta, breaks=50, probability=TRUE,
     main="Beta Distribution of p(t)",
     xlab="Probability of Infection",
     ylab="Density",
     col="lightblue",
     border="white")
curve(dnorm(x, mean=mean(p_beta), sd=sd(p_beta)), 
      add=TRUE, col="red", lwd=2)

par(mfrow=c(1,1)) # Reset plot layout

# Print summaries
cat("\n=== Normal Distribution Results ===")
summarize_distribution(v_norm, "ν (Normal)")
summarize_distribution(p_norm, "p(t) (Normal)")

cat("\n=== Beta Distribution Results ===")
summarize_distribution(v_beta, "ν (Beta)")
summarize_distribution(p_beta, "p(t) (Beta)")

## Question 2

In [ ]:
# Question 2 Data
n <- 200    # number of trials (dogs checked)
s <- 20     # number of non-compliant cases
n_samples <- 10000

# Calculate posterior parameters
posterior_params <- calculate_posterior_params(n, s)
posterior_samples <- rbeta(n_samples, posterior_params$alpha, posterior_params$beta)

# Create plots
par(mfrow=c(1,2))

# Plot 1: Prior (Uniform = Beta(1,1))
curve(dbeta(x, 1, 1), from=0, to=1,
      main="Prior Distribution\n(Uniform = Beta(1,1))",
      xlab="Probability of Non-Compliance (p)",
      ylab="Density",
      col="blue",
      lwd=2,
      ylim=c(0, 1.4))

# Plot 2: Posterior
hist(posterior_samples, freq=FALSE, breaks=30,
     main=sprintf("Posterior Distribution\nBeta(%d, %d)",
                 posterior_params$alpha, posterior_params$beta),
     xlab="Probability of Non-Compliance (p)",
     ylab="Density",
     col="lightblue", 
     border="white",
     xlim=c(0.05, 0.20),    
     ylim=c(0, 20))         

curve(dbeta(x, posterior_params$alpha, posterior_params$beta),
      add=TRUE, col="red", lwd=2,
      from=0.05, to=0.20)

par(mfrow=c(1,1))

# Calculate summary statistics
mode <- calculate_beta_mode(posterior_params$alpha, posterior_params$beta)
ci <- qbeta(c(0.025, 0.975), posterior_params$alpha, posterior_params$beta)

# Print summary statistics
cat(sprintf("\nPosterior Distribution (Beta(%d, %d)) Summary:\n", 
           posterior_params$alpha, posterior_params$beta))
cat("Mean:", round(mean(posterior_samples), 4), "\n")
cat("Median:", round(median(posterior_samples), 4), "\n")
if(!is.na(mode)) cat("Mode:", round(mode, 4), "\n")
cat("95% Credible Interval:", round(ci[1], 4), "to", round(ci[2], 4), "\n")

## Question 3

In [ ]:
# Question 3 Data
n_samples <- 10000
survey_2010 <- list(
    n = 200,  # number of trials
    s = 20    # number of non-compliant
)
survey_2022 <- list(
    n = 26,   # new trials
    s = 1     # new non-compliant
)
plot_range <- c(0, 0.25)
n_points <- 1000

# Calculate posterior parameters
prior_params <- calculate_posterior_params(survey_2010$n, survey_2010$s)
posterior_params <- calculate_posterior_params(
    n = survey_2010$n + survey_2022$n, 
    s = survey_2010$s + survey_2022$s,
    prior_alpha = 1,
    prior_beta = 1
)

# Generate samples and sequence for plotting
posterior_samples <- rbeta(n_samples, posterior_params$alpha, posterior_params$beta)
x <- seq(plot_range[1], plot_range[2], length=n_points)

# Calculate densities
prior_density <- dbeta(x, prior_params$alpha, prior_params$beta)
posterior_density <- dbeta(x, posterior_params$alpha, posterior_params$beta)

# Calculate and scale likelihood
likelihood <- dbinom(survey_2022$s, size=survey_2022$n, prob=x)
likelihood_scaled <- likelihood * max(prior_density) / max(likelihood)

# Create plot
plot(x, prior_density, type="l", col="blue", lwd=2,
     main="Prior, Likelihood, and Posterior Distributions",
     xlab="Probability of Non-Compliance",
     ylab="Density",
     ylim=c(0, max(c(prior_density, posterior_density))),
     xlim=plot_range)

lines(x, likelihood_scaled, col="green", lwd=2)
lines(x, posterior_density, col="red", lwd=2)

legend("topright", 
      legend=c("Prior (2010)", "Likelihood (2022)", "Posterior"),
      col=c("blue", "green", "red"),
      lwd=2)

# Print summaries using shared summarize_distribution function
cat("\nPosterior Distribution Summary Statistics:")
summarize_distribution(posterior_samples, "Posterior")

# Calculate comparison statistics
mode <- calculate_beta_mode(posterior_params$alpha, posterior_params$beta)
ci <- qbeta(c(0.025, 0.975), posterior_params$alpha, posterior_params$beta)
prior_mean <- prior_params$alpha/(prior_params$alpha + prior_params$beta)
posterior_mean <- posterior_params$alpha/(posterior_params$alpha + posterior_params$beta)
prob_greater_10 <- mean(posterior_samples > 0.10)

cat("\nComparison of Estimates:\n")
cat("Prior Mean (from 2010):", round(prior_mean, 4), "\n")
cat("Posterior Mean (after 2022):", round(posterior_mean, 4), "\n")
cat("Change in estimate:", round(posterior_mean - prior_mean, 4), "\n")
cat("\nProbability that non-compliance rate > 10%:", round(prob_greater_10, 4), "\n")

## Question 4

In [ ]:
# Data for Question 4
treatment_times <- c(105, 73, 140, 53, 100, 39, 129, 41, 115, 55, 145, 89, 45, 
                    35, 7, 36, 29, 109, 70, 31, 76, 90, 64, 39, 76)

# Compliance window
lower_limit <- 24
upper_limit <- 120

# Bootstrap function
bootstrap_samples <- function(data, n_boot=10000) {
    n <- length(data)
    means <- numeric(n_boot)
    sds <- numeric(n_boot)
    probs_outside <- numeric(n_boot)
    
    for(i in 1:n_boot) {
        boot_sample <- sample(data, size=n, replace=TRUE)
        means[i] <- mean(boot_sample)
        sds[i] <- sd(boot_sample)
        prob_below <- pnorm(lower_limit, means[i], sds[i])
        prob_above <- 1 - pnorm(upper_limit, means[i], sds[i])
        probs_outside[i] <- prob_below + prob_above
    }
    
    return(list(means=means, sds=sds, probs_outside=probs_outside))
}

# Run bootstrap
boot_results <- bootstrap_samples(treatment_times)

# Create visualizations
par(mfrow=c(2,2))

# Plot 1: Bootstrap distribution of means
hist(boot_results$means, breaks=30, probability=TRUE,
     main="Bootstrap Distribution of Mean",
     xlab="Mean Treatment Time (hours)",
     ylab="Density",
     col="lightblue", border="white")
abline(v=mean(treatment_times), col="red", lwd=2)

# Plot 2: Bootstrap distribution of standard deviations
hist(boot_results$sds, breaks=30, probability=TRUE,
     main="Bootstrap Distribution of SD",
     xlab="Standard Deviation of Treatment Time (hours)",
     ylab="Density",
     col="lightblue", border="white")
abline(v=sd(treatment_times), col="red", lwd=2)

# Plot 3: Original data with normal fit
hist(treatment_times, breaks=10, probability=TRUE,
     main="Original Treatment Times",
     xlab="Treatment Time (hours)",
     ylab="Density",
     col="lightblue", border="white")
curve(dnorm(x, mean(treatment_times), sd(treatment_times)), 
      add=TRUE, col="red", lwd=2)
abline(v=c(lower_limit, upper_limit), col="blue", lty=2)

# Plot 4: Bootstrap distribution of probability outside window
hist(boot_results$probs_outside, breaks=30, probability=TRUE,
     main="Probability Outside 24-120 Hour Window",
     xlab="Probability",
     ylab="Density",
     col="lightblue", border="white")

par(mfrow=c(1,1))

# Print summaries using shared summarize_distribution function
summarize_distribution(boot_results$means, "Bootstrap Mean Treatment Time")
summarize_distribution(boot_results$sds, "Bootstrap Standard Deviation")
summarize_distribution(boot_results$probs_outside, "Probability Outside Window")

# Additional statistics
actual_outside <- mean(treatment_times < lower_limit | treatment_times > upper_limit)
cat("\nActual proportion outside window in sample:", round(actual_outside, 4), "\n")

## Question 5

In [ ]:
# Data for Question 5
n_samples <- 10000
infection_params <- list(
    alpha = 20,
    v_min = 0.00001,
    v_max = 0.0006,
    v_mode = 0.000187,
    mu = 0.078,
    t = 2
)
treatment_params <- list(
    alpha = 22,
    beta = 206
)
timing_params <- list(
    mean = 0.1982,
    sd = 0.0472
)

# Generate infection probability samples
beta_q1 <- infection_params$alpha * (1 - infection_params$v_mode)/infection_params$v_mode
p1_samples <- rbeta(n_samples, infection_params$alpha, beta_q1)
p1_samples <- infection_params$v_min + (infection_params$v_max - infection_params$v_min) * p1_samples
p1_samples <- calculate_p(p1_samples, infection_params$mu, infection_params$t)

# Generate treatment probability samples
p2_samples <- rbeta(n_samples, treatment_params$alpha, treatment_params$beta)

# Generate timing probability samples
p3_samples <- rnorm(n_samples, timing_params$mean, timing_params$sd)
p3_samples <- pmin(pmax(p3_samples, 0), 1)

# Calculate overall probability
overall_prob <- p1_samples * (p2_samples + p3_samples - p2_samples * p3_samples)

# Create visualization
par(mfrow=c(2,2))

# Plot individual components
hist(p1_samples, breaks=50, probability=TRUE,
     main="Distribution of P(Infection)",
     xlab="Probability",
     ylab="Density",
     col="lightblue", border="white")

hist(p2_samples, breaks=50, probability=TRUE,
     main="Distribution of P(No Treatment)",
     xlab="Probability",
     ylab="Density",
     col="lightblue", border="white")

hist(p3_samples, breaks=50, probability=TRUE,
     main="Distribution of P(Wrong Timing)",
     xlab="Probability",
     ylab="Density",
     col="lightblue", border="white")

hist(overall_prob, breaks=50, probability=TRUE,
     main="Distribution of Overall Risk",
     xlab="Probability of Infection upon Return",
     ylab="Density",
     col="lightblue", border="white")
lines(density(overall_prob), col="red", lwd=2)

par(mfrow=c(1,1))

# Print summaries using shared summarize_distribution function
summarize_distribution(overall_prob, "Overall Infection Probability")

# Additional probability thresholds
thresholds <- c(0.0001, 0.001, 0.01)
cat("\nProbability that infection risk exceeds:\n")
for(threshold in thresholds) {
    pct <- threshold * 100
    prob <- mean(overall_prob > threshold)
    cat(sprintf("%.3f%%: %.4f\n", pct, prob))
}

## Question 6

In [ ]:
# Multiple simulations to get distribution of introduction probability
n_dogs <- 1000
n_sims <- 10000  # Number of simulations for each probability calculation
n_iterations <- 10000  # Number of probability estimates to generate distribution

# Calculate probability of introduction multiple times
introduction_probs <- replicate(n_iterations, {
    # For each iteration, simulate 1000 dogs and calculate probability of at least one introduction
    yearly_sim <- rbinom(n_sims, size=n_dogs, prob=overall_prob)
    mean(yearly_sim > 0)  # Probability of at least one introduction
})

# Create visualization
hist(introduction_probs, 
     breaks=30,
     probability=TRUE,
     main="Distribution of Introduction Probability",
     xlab="Probability of At Least One Introduction",
     ylab="Density",
     col="lightblue",
     border="white")

# Add density curve
lines(density(introduction_probs), col="red", lwd=2)

# Print summary statistics
cat("\nDistribution of E. multilocularis Introduction Probability:\n")
cat("Mean probability:", round(mean(introduction_probs), 4), "\n")
cat("Median probability:", round(median(introduction_probs), 4), "\n")
cat("95% CI: [", 
    round(quantile(introduction_probs, 0.025), 4), ", ", 
    round(quantile(introduction_probs, 0.975), 4), "]\n")

## Question 7

In [ ]:
# Create scenarios for sensitivity analysis
v_scenarios <- c(0.0001, 0.000187, 0.0003, 0.0004, 0.0005)
n_dogs_scenarios <- c(500, 1000, 2000, 3000, 5000)

# Create matrices to store results
mean_matrix <- matrix(NA, nrow=length(v_scenarios), ncol=length(n_dogs_scenarios))
prob_matrix <- matrix(NA, nrow=length(v_scenarios), ncol=length(n_dogs_scenarios))

# Calculate results
for(i in seq_along(v_scenarios)) {
    for(j in seq_along(n_dogs_scenarios)) {
        p1 <- calculate_p(v_scenarios[i], mu, t)
        p2 <- posterior_mean
        p3 <- mean(p3_samples)
        overall_prob <- p1 * (p2 + p3 - p2 * p3)
        annual_infections <- rbinom(n_samples, size=n_dogs_scenarios[j], prob=overall_prob)
        mean_matrix[i,j] <- mean(annual_infections)
        prob_matrix[i,j] <- mean(annual_infections > 0)
    }
}

# Part 1: Effect of Infection Pressure (ν)
par(mfrow=c(2,2))

# 1. Linear relationship for different dog populations
matplot(v_scenarios, mean_matrix, type="b", pch=16,
        col=rainbow(length(n_dogs_scenarios)),
        main="Effect of Infection Pressure\non Mean Annual Infections",
        xlab="Infection Pressure (ν)",
        ylab="Mean Annual Infections")

# 2. Log-transformed relationship
matplot(log10(v_scenarios), log10(mean_matrix), type="b", pch=16,
        col=rainbow(length(n_dogs_scenarios)),
        main="Log-Log Plot of Infection Pressure Effect",
        xlab="log10(Infection Pressure)",
        ylab="log10(Mean Annual Infections)")

# 3. Probability of at least one infection vs ν
matplot(v_scenarios, prob_matrix, type="b", pch=16,
        col=rainbow(length(n_dogs_scenarios)),
        main="Effect of Infection Pressure\non Introduction Probability",
        xlab="Infection Pressure (ν)",
        ylab="Probability of ≥1 Introduction")

# 4. Relative change from baseline
baseline_v <- 0.000187
relative_change <- sweep(mean_matrix, 2, mean_matrix[which(v_scenarios == baseline_v),], "/")
matplot(v_scenarios, relative_change, type="b", pch=16,
        col=rainbow(length(n_dogs_scenarios)),
        main="Relative Change from Baseline\n(ν = 0.000187)",
        xlab="Infection Pressure (ν)",
        ylab="Relative Change in Mean Infections")
abline(h=1, lty=2, col="gray")
legend("bottomright", legend=paste(n_dogs_scenarios, "dogs"),
       col=rainbow(length(n_dogs_scenarios)), pch=16, lty=1,
       cex=0.8, bg="white")

# Part 2: Effect of Number of Dogs
par(mfrow=c(2,2))

# 1. Linear relationship for different infection pressures
matplot(n_dogs_scenarios, t(mean_matrix), type="b", pch=16,
        col=rainbow(length(v_scenarios)),
        main="Effect of Number of Dogs\non Mean Annual Infections",
        xlab="Number of Dogs",
        ylab="Mean Annual Infections")
legend("topleft", legend=paste("ν =", formatC(v_scenarios, format="e", digits=1)),
       col=rainbow(length(v_scenarios)), pch=16, lty=1,
       cex=0.7, bg="white")

# 2. Log-transformed relationship
matplot(log10(n_dogs_scenarios), log10(t(mean_matrix)), type="b", pch=16,
        col=rainbow(length(v_scenarios)),
        main="Log-Log Plot of Number of Dogs Effect",
        xlab="log10(Number of Dogs)",
        ylab="log10(Mean Annual Infections)")

# 3. Probability of at least one infection vs number of dogs
matplot(n_dogs_scenarios, t(prob_matrix), type="b", pch=16,
        col=rainbow(length(v_scenarios)),
        main="Effect of Number of Dogs\non Introduction Probability",
        xlab="Number of Dogs",
        ylab="Probability of ≥1 Introduction")

# 4. Relative change from baseline
baseline_n <- 1000
relative_change <- sweep(t(mean_matrix), 1, t(mean_matrix)[,which(n_dogs_scenarios == baseline_n)], "/")
matplot(n_dogs_scenarios, relative_change, type="b", pch=16,
        col=rainbow(length(v_scenarios)),
        main="Relative Change from Baseline\n(1000 dogs)",
        xlab="Number of Dogs",
        ylab="Relative Change in Mean Infections")
abline(h=1, lty=2, col="gray")

# Reset plot layout
par(mfrow=c(1,1))

# Print summary statistics for number of dogs effect
cat("\nEffect of Number of Dogs:\n")
# Calculate elasticity for each infection pressure
for(i in seq_along(v_scenarios)) {
    model <- lm(log10(mean_matrix[i,]) ~ log10(n_dogs_scenarios))
    cat(sprintf("\nFor ν = %e:\n", v_scenarios[i]))
    cat("Elasticity (log-log slope):", round(coef(model)[2], 4), "\n")
    cat("R-squared:", round(summary(model)$r.squared, 4), "\n")
}